In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# Load Wine dataset
wine = load_wine()
X, y = wine.data, wine.target

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Define number of RBF centers (equal to number of clusters)
k = 10  # Number of RBF neurons (tunable hyperparameter)

# Perform K-Means clustering to find centers
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(X_train.numpy())
centroids = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32)

# Compute standard deviation (spread of RBF neurons)
def compute_sigma(centroids):
    pairwise_dists = cdist(centroids, centroids, 'euclidean')
    sigma = np.mean(pairwise_dists)  # Use mean distance as spread
    return sigma

sigma = compute_sigma(centroids)

# Define RBF Network
class RBFNetwork(nn.Module):
    def __init__(self, num_centroids, input_dim, output_dim):
        super(RBFNetwork, self).__init__()
        self.centroids = nn.Parameter(centroids)  # Centers (weights for hidden layer)
        self.sigma = torch.tensor(sigma, dtype=torch.float32, requires_grad=False)  # Fixed sigma
        self.fc = nn.Linear(num_centroids, output_dim)  # Fully connected output layer

    def gaussian_rbf(self, x):
        """
        Compute Gaussian RBF activations.
        x: (batch_size, input_dim)
        Returns: (batch_size, num_centroids)
        """
        # Compute distance between input and each centroid
        diff = x.unsqueeze(1) - self.centroids.unsqueeze(0)  # (batch_size, num_centroids, input_dim)
        dist = torch.norm(diff, dim=2)  # Euclidean distance
        return torch.exp(- (dist ** 2) / (2 * self.sigma ** 2))

    def forward(self, x):
        rbf_out = self.gaussian_rbf(x)  # Hidden layer output
        return torch.softmax(self.fc(rbf_out), dim=1)  # Softmax for classification

# Initialize the model
num_classes = len(set(y_train.numpy()))
model = RBFNetwork(k, X_train.shape[1], num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.01)  # Only train output layer

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).float().mean().item() * 100

print(f'Accuracy of RBF Network: {accuracy:.2f}%')


Epoch [10/50], Loss: 1.0767
Epoch [20/50], Loss: 1.0379
Epoch [30/50], Loss: 1.0015
Epoch [40/50], Loss: 0.9659
Epoch [50/50], Loss: 0.9326
Accuracy of RBF Network: 91.67%


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import pandas as pd
# Load Olive Oil dataset
df= pd.read_csv("/content/olive.csv")
X = df.drop("Region",axis = 1)
y = df["Region"]

# Encode target labels as integers
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define number of RBF centers (equal to number of clusters)
k = 10  # Number of RBF neurons

# Perform K-Means clustering to find centers
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(X_train.numpy())
centroids = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32)

# Compute standard deviation (spread of RBF neurons)
def compute_sigma(centroids):
    pairwise_dists = cdist(centroids, centroids, 'euclidean')
    sigma = np.mean(pairwise_dists)  # Use mean distance as spread
    return sigma

sigma = compute_sigma(centroids)

# Define the RBF Network
class RBFNetwork(nn.Module):
    def __init__(self, num_centroids, input_dim, output_dim):
        super(RBFNetwork, self).__init__()
        self.centroids = nn.Parameter(centroids)  # Centers (weights for hidden layer)
        self.sigma = sigma
        self.fc = nn.Linear(num_centroids, output_dim)  # Output layer (fully connected)

    def gaussian_rbf(self, x):
        """
        Compute Gaussian RBF activations.
        x: (batch_size, input_dim)
        Returns: (batch_size, num_centroids)
        """
        # Compute distance between input and each centroid
        diff = x.unsqueeze(1) - self.centroids.unsqueeze(0)  # (batch_size, num_centroids, input_dim)
        dist = torch.norm(diff, dim=2)  # Euclidean distance
        return torch.exp(- (dist ** 2) / (2 * self.sigma ** 2))

    def forward(self, x):
        rbf_out = self.gaussian_rbf(x)  # Hidden layer output
        return self.fc(rbf_out)  # Output layer

# Initialize the model
num_classes = len(np.unique(y_train.numpy()))
model = RBFNetwork(k, X_train.shape[1], num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).float().mean().item() * 100

print(f'Accuracy of RBF Network on Olive Oil Dataset: {accuracy:.2f}%')


Epoch [10/150], Loss: 1.9725
Epoch [20/150], Loss: 1.7841
Epoch [30/150], Loss: 1.6637
Epoch [40/150], Loss: 1.5522
Epoch [50/150], Loss: 1.4536
Epoch [60/150], Loss: 1.3687
Epoch [70/150], Loss: 1.2960
Epoch [80/150], Loss: 1.2333
Epoch [90/150], Loss: 1.1783
Epoch [100/150], Loss: 1.1294
Epoch [110/150], Loss: 1.0852
Epoch [120/150], Loss: 1.0449
Epoch [130/150], Loss: 1.0076
Epoch [140/150], Loss: 0.9730
Epoch [150/150], Loss: 0.9407
Accuracy of RBF Network on Olive Oil Dataset: 74.78%
